<a href="https://colab.research.google.com/github/MarvelManueke/Machine-Learning-Projects/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
#!pip install numpy==1.24
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

#print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label','message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label','message'])

train_data['label']=train_data['label'].apply(lambda x: x.replace("ham","0"))
train_data['label']=train_data['label'].apply(lambda x: x.replace("spam","1"))
train_data.tail()

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

def remove_stopwords(text):
    text = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word.isalpha() and not word in stop_words]
    return ' '.join(text)

x = train_data.apply(lambda row: remove_stopwords(row['message']), axis=1)
y = train_data['label'].astype(np.int64)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_words = 20000
max_length = 500

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
x_train = pad_sequences(sequences, maxlen=max_length)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(input_dim = max_words, output_dim=4, input_length=max_length))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train, y, validation_split=0.2, epochs=5, batch_size=20)

In [ ]:
cleaned_text = remove_stopwords('Why pay more for expensive meds when you can order them online and save $$$?')
sequence = tokenizer.texts_to_sequences([cleaned_text])
padded_sequence = pad_sequences(sequence, maxlen=max_length)
model.predict(padded_sequence)[0][0]

In [ ]:
tf.keras.utils.disable_interactive_logging()
def predict_message(message):
    # Reshape the encoded message to match the model's input shape.
    cleaned_text = remove_stopwords(message)
    sequence = tokenizer.texts_to_sequences([cleaned_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)
    # Make the prediction using the trained model.
    prediction = model.predict(padded_sequence)[0][0]

    # Determine the label based on the prediction.
    if prediction >= 0.5:
        label = "spam"
    else:
        label = "ham"

    # Return the prediction and label in a list.
    return [prediction, label]

message = "i dont want to go. can we try it a different day? available sat"
predict_message(message)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
